In [1]:
import os
import re
from docx import Document
from docx.shared import Cm, Pt
from docx.enum.text import WD_ALIGN_PARAGRAPH

In [63]:
# Устойчивость
def stability(f):
    regulator = f[:4].upper()
    
    ramp = [""]
    if re.search(r'[+-]\d+%', f):
        ramp = re.search(r'[+-]\d+%', f)
        
    time = [""]
    if re.search(r'\d+с', f):
        time = re.search(r'\d+с', f)
        
    result = regulator + ", проверка устойчивости," + " толчок " + ramp[0] + " " + time[0].replace("с", " секунд")
    
    if f.find("руч") > -1:
        result += " в ручном режиме"
    else:
        if f.find("pss=on") > -1 or f.find("с псс") > -1:
            result += " в автоматическом режиме, PSS введен"
        elif f.find("pss=off") > -1 or f.find("без псс") > -1:
            result += " в автоматическом режиме, PSS выведен"
    return result

def forcing(f):
    regulator = f[:4].upper()
    
    ramp = [""]
    if re.search(r'[+-]\d+%', f):
        ramp = re.search(r'[+-]\d+%', f)
        
    time = [""]
    if re.search(r'\d+с', f):
        time = re.search(r'\d+с', f)
        
    limiter = [""]
    if re.search(r'631=\d[.,]\d+', f):
        limiter = re.search(r'631=\d[.,]\d+', f)
        
    result = regulator + ", опыт ФОРСИРОВКИ, ограничитель: SetLimIfMax " + limiter[0] + ","
    if time[0] != "" or ramp[0] != "":
        result += " толчок: " + ramp[0] + " " + time[0].replace("с", " секунд")

    return result

# ОМВ
def uel(f):
    regulator = f[:4].upper()
    
    ramp = [""]
    if re.search(r'[+-]\d+%', f):
        ramp = re.search(r'[+-]\d+%', f)
        
    time = [""]
    if re.search(r'\d+с', f):
        time = re.search(r'\d+с', f)
        
    result = regulator + ", проверка ОМВ,"
    if time[0] != "" or ramp[0] != "":
        result += " толчок " + ramp[0] + " " + time[0].replace("с", " секунд")
        if f.find("pss=on") > -1 or f.find("с псс") > -1:
            result += ", PSS введен"
        elif f.find("pss=off") > -1 or f.find("без псс") > -1:
            result += ", PSS выведен"
    elif f.find("плавно") > -1:
        result += " плавное вступлении"

    return result


# ОПС
def ogl(f):
    regulator = f[:4].upper()

    return regulator + ", проверка ОПС"


# ОПР
def oel(f):
    regulator = f[:4].upper()

    return regulator + ", проверка ОПР"


# ОМТР
def omtr(f):
    regulator = f[:4].upper()
    
    ramp = [""]
    if re.search(r'[+-]\d+%', f):
        ramp = re.search(r'[+-]\d+%', f)
        
    time = [""]
    if re.search(r'\d+с', f):
        time = re.search(r'\d+с', f)

    param = 1
    limiter = [""]
    if re.search(r'631=\d[.,]\d+', f):
        limiter = re.search(r'631=\d[.,]\d+', f)
        param = 631
    if re.search(r'632=\d[.,]\d+', f):
        limiter = re.search(r'632=\d[.,]\d+', f)
        param = 632

    result = ""
    if param == 631:
        result = regulator + ", проверка ОМТР, ограничитель: SetLimIfMax " + limiter[0] + ","
    elif param == 632:
        result = regulator + ", проверка запрета форсировки (If=1), ограничитель: SetLimIfMaxOff  " + limiter[0] + ","

    if time[0] != "" or ramp[0] != "":
        result += " толчок: " + ramp[0] + " " + time[0].replace("с", " секунд")
    elif f.find("плавно") > -1:
        result += " плавное вступлении"

    return result


# Переводы
def transfer(f):
    regulator = f[:4].upper()
    result = ""
    #return regulator + "," + f[4:-4].replace("пер", "перевод").replace("авт", "автоматический").replace("руч", "ручной")
    if f.find("с авт на руч") > -1:
        result = regulator + "," + f[4:-4].replace("пер", "перевод").replace("авт", "автоматического режима (Ug)").replace("руч", "ручной режим (If)")
    elif f.find("с руч на авт") > -1:
        result = regulator + "," + f[4:-4].replace("пер", "перевод").replace("авт", "автоматический режим (Ug)").replace("руч", "ручного режима (If)")
    else:
        result = regulator + "," + f[4:-4].replace("пер", "перевод").replace("авт", "автоматический режим (Ug)").replace("руч", "ручного режима (If)")
    return result

# Сгон реактивки к 0
def q0(f):
    regulator = f[:4].upper()
    return regulator + ", (Q->0) разгрузка генератора по реактивной мощности CosPhi = 1"

def net(f):
    regulator = f[:4].upper()
    return regulator + ", включение в сеть"

In [72]:
def parsePowerFilder(power):
    result = {}
    for root, dirs, files in os.walk(power):
        power_string = "Активная мощность: " + power + " [МВт], ";
        for f in files:
            name_pictures = ""
            if f.find('уст') > -1:
                name_pictures = stability(f)
            elif f.find('форс') > -1:
                name_pictures = forcing(f)
            elif f.find('омв') > -1:
                name_pictures = uel(f)
            elif f.find('опр') > -1:
                name_pictures = oel(f)
            elif f.find('омтр') > -1:
                name_pictures = omtr(f)
            elif f.find('пер') > -1:
                name_pictures = transfer(f)
            elif f.find('q-0') > -1:
                name_pictures = q0(f)
            elif f.find('опс') > -1:
                name_pictures = ogl(f)
            elif f.find('вкл в сеть') > -1:
                name_pictures = net(f)
            else:
                continue
            result.update({root + "/" + f: power_string + name_pictures})

    return result

In [50]:
def genPicturesList():
    result = {}
    for power in ['0', '10', '40', '70', '100', '130', '170', '180', '205']:
        result.update(parsePowerFilder(power))
    
    return result

In [77]:
for i, k in enumerate(genPicturesList().keys()):
    print(i+1, genPicturesList()[k], sep='\t')

1	Активная мощность: 0 [МВт], АРВ1, включение в сеть
2	Активная мощность: 10 [МВт], АРВ1, проверка ОМВ, плавное вступлении
3	Активная мощность: 10 [МВт], АРВ2, проверка ОМВ, плавное вступлении
4	Активная мощность: 40 [МВт], АРВ1, проверка ОМВ, плавное вступлении
5	Активная мощность: 40 [МВт], АРВ2, проверка ОМВ, плавное вступлении
6	Активная мощность: 70 [МВт], АРВ1, проверка ОМВ, толчок -5% 15 секунд, PSS выведен
7	Активная мощность: 70 [МВт], АРВ1, проверка ОМВ, плавное вступлении
8	Активная мощность: 70 [МВт], АРВ1, перевод с арв1 на арв2 автоматический режим (Ug)
9	Активная мощность: 70 [МВт], АРВ1, перевод с арв2 на арв1 автоматический режим (Ug)
10	Активная мощность: 70 [МВт], АРВ1, проверка устойчивости, толчок +5% 15 секунд в автоматическом режиме, PSS выведен
11	Активная мощность: 70 [МВт], АРВ1, проверка устойчивости, толчок +5% 15 секунд в автоматическом режиме, PSS введен
12	Активная мощность: 70 [МВт], АРВ1, проверка устойчивости, толчок -5% 15 секунд в ручном режиме
13	Ак

In [78]:
document = Document()

for section in document.sections:
    section.top_margin = Cm(1)
    section.bottom_margin = Cm(1)
    section.left_margin = Cm(1.2)
    section.right_margin = Cm(1.2)

style = document.styles['Normal']
font = style.font
font.name = 'Times New Roman'
font.size = Pt(10)

p = document.add_paragraph('Осциллограммы опытов при работе турбогенератора ТГ2 в сети')
document.add_page_break()
p.style = document.styles['Normal']
p.alignment = WD_ALIGN_PARAGRAPH.CENTER

pic_list = []
result = genPicturesList()
for i, fileName in enumerate(result.keys()):
    pic = document.add_picture(fileName, width=Cm(19))
    pic.style = document.styles['Normal']
    pic.alignment = WD_ALIGN_PARAGRAPH.CENTER
    pic_list.append(pic)
    
    document.add_heading("Рисунок 2.{}. ".format(i+1) + result[fileName] + ".", level=3)
    document.add_page_break()
    #r.add_break()

document.save('Сеть.docx')